In [3]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import math
import timeit

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
def read_data(city_name):
    trip_data = pd.read_csv("Project_Data/Combined_Data/"+city_name+".csv", encoding = "ISO-8859-1")
    trip_data["trip_duration"] = pd.to_timedelta(trip_data["trip_duration"])
    trip_data["datetime_start"] = pd.to_datetime(trip_data["datetime_start"])
    trip_data["datetime_end"] = pd.to_datetime(trip_data["datetime_end"])
    trip_data["day"] = pd.to_datetime(trip_data["day"])
    return trip_data

In [5]:
Duesseldorf_set = read_data("duesseldorf")
Bremen_set = read_data("bremen")

## Spielen:

In [19]:
def plot_cluster():
    plt.figure(figsize=(12,8))
    plt.scatter(Duesseldorf_set[Duesseldorf_set["distance"]]['temperature'], Duesseldorf_set["distance"]['temperature'], marker='x', color='C0')
    plt.xlim([0,2600])
    plt.ylim([0,0.21])
    plt.xlabel("Mean Area")
    plt.ylabel("Mean Concave Points")
    plt.legend(['Malignant', 'Benign'])
    
plot_cluster()

KeyError: "None of [Float64Index([ 2.6629578210697096,   2.182597844114829,  1.2564863871006955,\n               3.8833016980883013,  1.1033492930621034,  1.2960095360373924,\n               1.6293593436497005,   2.985949419651796, 0.30314801497127397,\n               0.6991414206362674,\n              ...\n               1.8418517209080358,  2.0000110908923507,   2.190499476470145,\n                0.841057027560746,  0.1961324003726801, 0.08432599487263265,\n               0.4591566173235575, 0.01792001089731299,   2.383757589871979,\n               1.5357985679178845],\n             dtype='float64', length=18038)] are in the [columns]"

<Figure size 864x576 with 0 Axes>